<a href="https://colab.research.google.com/github/sahilk45/PyTorch-1Go/blob/main/RNN_bases_QnA_system_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

df=pd.read_csv("/content/100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [3]:
#tokenize
def tokenize(text):
  text=text.lower()
  text=text.replace('?','')
  text=text.replace("'","")
  return text.split()

In [4]:
tokenize("Who wrote 'To Kill a Mockingbird'?")

['who', 'wrote', 'to', 'kill', 'a', 'mockingbird']

In [5]:
#vocab - how many unique words in data:
vocab={'<UNK>':0}

In [6]:
def build_vocab(row):
  tokenize_question=tokenize(row['question'])
  tokenize_answer=tokenize(row['answer'])

  merge_tokens=tokenize_question+tokenize_answer

  for token in merge_tokens:
    if token not in vocab:
      vocab[token]=len(vocab)

In [7]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [9]:
len(vocab)

324

In [10]:
#convert words to numerical indices:
def text_to_indices(text,vocab):
  indexed_text=[]

  for text in tokenize(text):
    if text in vocab:
      indexed_text.append(vocab[text])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [11]:
text_to_indices("what is your name?",vocab)

[1, 2, 0, 0]

In [12]:
import torch
from torch.utils.data import Dataset,DataLoader

In [15]:
class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df=df
    self.vocab=vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index):
    numerical_question=text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer=text_to_indices(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(numerical_question),torch.tensor(numerical_answer)

In [16]:
dataset=QADataset(df,vocab)

In [17]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [18]:
dataloader=DataLoader(dataset,batch_size=1,shuffle=True) #Also due to this no padding req beac it is req when we are processing more that 1 data in 1 batch and shape of data is different

In [19]:
for question,answer in dataloader:
  print(question,answer)

tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([[124]])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([[205]])
tensor([[ 1,  2,  3, 24, 25,  5, 26, 19, 27]]) tensor([[28]])
tensor([[ 42,  18, 118,   3, 186, 187]]) tensor([[188]])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([[215]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[ 42, 250, 251, 118, 252, 253]]) tensor([[254]])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([[58]])
tensor([[ 1,  2,  3, 50, 51, 19,  3, 45]]) tensor([[52]])
tensor([[  1,   2,   3,  17, 115,  83,  84]]) tensor([[116]])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([[170]])
tensor([[ 1,  2,  3, 33, 34,  5, 35]]) tensor([[36]])
tensor([[  1,   2,   3, 234,   5, 235]]) tensor([[131]])
tensor([[ 10,  29, 130, 131]]) tensor([[132]])
tensor([[  1,   2,   3,   4,   5, 206]]) tensor([[207]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([[31

RNN Architecture

In [20]:
import torch.nn as nn

In [50]:
class SimpleRNN(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size,embedding_dim=50) #every word in 50 dim conversion
    self.rnn = nn.RNN(50,64,batch_first=True)
    self.fc = nn.Linear(64,vocab_size)  #Not wrapping in sequential beac RNN gives two outputs one-all hidden statesfrom O1 to O6 and second-different O6 which our final output but sequential layer only expects 1 output from each laye(nn.Sequential). Therefore manual code for forward pass

  def forward(self,x):
    embedded_question = self.embedding(x) # Changed 'question' to 'x' for consistency with argument name
    # The RNN expects (batch, sequence, feature) when batch_first=True
    hidden_output, final_hidden_state = self.rnn(embedded_question)
    # The final_hidden_state has shape (num_layers * num_directions, batch_size, hidden_size)
    # For a simple RNN, num_layers=1, num_directions=1, so shape is (1, batch_size, hidden_size)
    # We need to squeeze the first dimension and then pass it to the linear layer
    output = self.fc(final_hidden_state.squeeze(0)) #batch dim also coming in output

    return output

In [35]:
dataset[0][0] #8vectors

tensor([1, 2, 3, 4, 5, 6])

In [30]:
x = nn.Embedding(324,embedding_dim=50)

In [36]:
a = x(dataset[0][0]).shape #each vector of 50 dim

In [37]:
y=nn.RNN(50,64)

In [39]:
example_input_indices = dataset[0][0]
embedded_tensor = x(example_input_indices)
rnn_input = embedded_tensor.unsqueeze(1)
output, hidden = y(rnn_input)

print(f"RNN Input shape: {rnn_input.shape}")
print(f"RNN Output shape: {output.shape}")
print(f"RNN Hidden state shape: {hidden.shape}")

RNN Input shape: torch.Size([6, 1, 50])
RNN Output shape: torch.Size([6, 1, 64])
RNN Hidden state shape: torch.Size([1, 1, 64])


In [44]:
learnin_rate=0.001
epochs=20

In [51]:
model=SimpleRNN(len(vocab))

In [52]:
creterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learnin_rate)

In [54]:
#training loop:
for epoch in range(epochs):
  total_loss=0
  for question,answer in dataloader:
    optimizer.zero_grad()

    #forward pass:
    output = model(question)

    # The target tensor 'answer' has shape (batch_size, sequence_length_of_answer).
    # For CrossEntropyLoss with (N, C) input, the target should be (N)
    # containing class indices.
    # Since our answers are single words, and batch_size=1, 'answer' is (1, 1).
    # We need to squeeze the last dimension to get it to (1).
    target_answer = answer.squeeze(1)

    loss = creterion(output, target_answer)

    #backprop:
    loss.backward()

    #update:
    optimizer.step()

    total_loss+=loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss/len(dataloader)}")

Epoch: 1, Loss: 5.852680185106066
Epoch: 2, Loss: 5.059383005566067
Epoch: 3, Loss: 4.166187281078763
Epoch: 4, Loss: 3.4929864830440946
Epoch: 5, Loss: 2.9150102270974054
Epoch: 6, Loss: 2.3731688314014012
Epoch: 7, Loss: 1.8915604929129282
Epoch: 8, Loss: 1.4578585969077216
Epoch: 9, Loss: 1.122229395972358
Epoch: 10, Loss: 0.8497591892878215
Epoch: 11, Loss: 0.6576881176895566
Epoch: 12, Loss: 0.513031266298559
Epoch: 13, Loss: 0.4106296204858356
Epoch: 14, Loss: 0.33582898212803736
Epoch: 15, Loss: 0.2773181983166271
Epoch: 16, Loss: 0.23377272089322407
Epoch: 17, Loss: 0.19890326104230352
Epoch: 18, Loss: 0.17028884722126855
Epoch: 19, Loss: 0.14791633267369536
Epoch: 20, Loss: 0.12866126915646925


Prediction

In [57]:
def predict(model,question,threshold=0.5):
  #convert que to num:
  numerical_question=text_to_indices(question,vocab)

  #convert que to tensor:
  # Corrected: unsqueeze(0) to get (1, sequence_length) for batch_first=True RNN
  question_tensor=torch.tensor(numerical_question).unsqueeze(0)

  #send to model: will get 324 logits which we assume as probabilities
  output=model(question_tensor)

  #convert logits to prob:
  probs = torch.nn.functional.softmax(output,dim=1)

  #find index of max prob
  value, index = torch.max(probs,dim=1)

  if value.item()<threshold:
    print("Sorry, I don't know this based on my knowledge!")
  else:
    # Corrected: use .item() to extract scalar index for list indexing
    print(list(vocab.keys())[index.item()])

In [58]:
predict(model,"What is capital of france?")

paris


In [59]:
predict(model,'What is largest planet in our solar system?')

jupiter
